Notebook to generate data using folktables.

In [ ]:
from folktables import ACSDataSource, ACS

In [ ]:
data_source = ACSDataSource(survey_year='2023', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=['MI'], download=True)
print(type(acs_data))

<class 'pandas.core.frame.DataFrame'>


In [5]:
acs_data.to_csv('MI_2023_person.csv')

Selected Features

AGEP: Age

COW: Class of Worker

SCHL: Educational Attainment

MAR: Marital Status

OCCP: Occupation

POBP: Place of Birth

RELP: Relationship to Reference Person

WKHP: Usual Hours Worked Per Week Past 12 Months

SEX: Sex

RAC1P: Recoded Detailed Race Code


In [ ]:
# Columns to keep
columns_to_keep = ['AGEP', 'COW', 'SCHL', 'MAR', 'PUMA', 'OCCP', 'POBP', 'WKHP', 'SEX', 'RAC1P']

# Filter the DataFrame
filtered_df = acs_data[columns_to_keep]

print(filtered_df)

        AGEP  COW  SCHL  MAR  PUMA  REGION    OCCP  POBP  WKHP  SEX  RAC1P
0         20  1.0  19.0    5  2600       2  2002.0    26  50.0    1      1
1         71  NaN  20.0    1   100       2     NaN    26   NaN    2      1
2         27  NaN  18.0    5   500       2     NaN    26   NaN    1      2
3         18  1.0  18.0    5  1801       2  7750.0    26  30.0    1      1
4         94  NaN  18.0    2   400       2     NaN    26   NaN    2      1
...      ...  ...   ...  ...   ...     ...     ...   ...   ...  ...    ...
102576    71  NaN  21.0    1  3002       2     NaN    40   NaN    2      1
102577    76  NaN  19.0    1  3205       2     NaN    42   NaN    2      1
102578    85  NaN  16.0    1  3205       2     NaN    42   NaN    1      1
102579    63  2.0  16.0    1  2800       2   420.0    26  40.0    2      1
102580    59  1.0  16.0    1  2800       2   650.0    26  40.0    1      1

[102581 rows x 11 columns]


Check racial diversity in data

In [8]:
# Number of unique values
num_unique_values = filtered_df['RAC1P'].nunique()
print(f"Number of unique values in RAC1P: {num_unique_values}")

# Counts of each unique value
value_counts = filtered_df['RAC1P'].value_counts()
print("\nCounts of each unique value in RAC1P:")
print(value_counts)

# Proportions of each unique value
value_proportions = filtered_df['RAC1P'].value_counts(normalize=True)
print("\nProportions of each unique value in RAC1P:")
print(value_proportions)

Number of unique values in RAC1P: 9

Counts of each unique value in RAC1P:
1    83277
2     7459
9     6612
6     2700
8     1657
3      716
5      128
7       27
4        5
Name: RAC1P, dtype: int64

Proportions of each unique value in RAC1P:
1    0.811817
2    0.072713
9    0.064456
6    0.026321
8    0.016153
3    0.006980
5    0.001248
7    0.000263
4    0.000049
Name: RAC1P, dtype: float64


Aggregate PUMAs into to n regions

In [14]:
# Parameterizable number of groups
n = 3

# Step 1: Count occurrences of each PUMA
puma_counts = filtered_df['PUMA'].value_counts()

# Step 2: Divide PUMAs into approximately equal-sized groups
grouped_pumas = {}
group_size = len(filtered_df) // n
current_group = 1
current_count = 0

for puma, count in puma_counts.items():
    if current_count + count > group_size and current_group < n:
        current_group += 1
        current_count = 0
    grouped_pumas[puma] = current_group
    current_count += count

# Step 3: Create the new feature
filtered_df['Aggregated_PUMA'] = filtered_df['PUMA'].map(grouped_pumas)

# Check the result
# print(filtered_df)
print("\nCounts of each group in Aggregated_PUMA:")
print(filtered_df['Aggregated_PUMA'].value_counts())


Counts of each group in Aggregated_PUMA:
3    36327
2    34065
1    32189
Name: Aggregated_PUMA, dtype: int64


C:\Users\prana\AppData\Local\Temp\ipykernel_23232\123881483.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Aggregated_PUMA'] = filtered_df['PUMA'].map(grouped_pumas)


In [16]:
filtered_df.to_csv('MI_2023_agg_3_regions.csv')